You have different tasks to finish within the day. 7 am is the earliest time and 10 pm is the latest time of the day to finish all the tasks. Since each time block is 30', there are (15 hours)*(2 blocks/hour)= 30 blocks. Each task takes different amount of time to complete and different important score. Given that there are scheduled meetings during the day that you cannot assign the tasks.  How do you assign the tasks to maximize the productivty? Note: Since there is limited time per day, you don't need to assign every task in one day, the goal is not to fit every task but to maximize the total score

Input parameters: number of tasks (n), number of time blocks (B), important score of a task (s[i]), time for a task (t[i]), availability of a block (a[b]), with a[b]=0 if not available and a[b]=1 if available

Decision Variables: Whether or not to assign the task in a specific block of time
 
Constraints:
-One task per block
-Time performing all tasks should be less than the time available
-Do not need to be consecutive

In [1]:
import pandas as pd
tasks = pd.read_excel('Tasks.xlsx', 'Tasks')
tasks

,Task name,Important score (1-5),Time to finish,Num of blocks
0,Complete math homework,4,60,2
1,Study for the Physics quiz,5,120,4
2,Coffee with Taylor,1,60,2
3,Read book,3,30,1
4,Complete programming project,3,150,5
5,Reply to emails,2,30,1
6,Finish essay,2,120,4
7,Watch movie,1,90,3
8,Call mom,2,30,1


In [2]:
schedule = pd.read_excel('Tasks.xlsx','Schedule',usecols="B",header=None)
schedule


,1
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1
8,1
9,0


In [15]:
s = list(tasks['Important score (1-5)'])

d = list(tasks['Num of blocks'])

b = list(schedule.iloc[:,0])

In [16]:
B = len(b)
n = len(s)

In [17]:
#Time blocks available
A = sum(b)
A

14

In [18]:
from pulp import *

In [34]:
prob = LpProblem("Schedule_Tasks",LpMaximize)

In [35]:
#Define variable
y = LpVariable.dicts('Block', [(i,t) for i in range(n) for t in range(B)],
                    cat='Binary')

In [36]:
#Define objective
prob += lpSum(s[i]*b[t]*y[(i,t)] for i in range(n) for t in range(B))

In [37]:
#Define constraints

#Sum of the time bocks of the assigned tasks should be not greater than the number of available time blocks
prob += lpSum(y[(i,t)] for i in range(n) for t in range(B)) <= A

#Total number of time blocks assigned for each task should be less than or equal to the time needed to finish the task
for i in range(n):
    prob += lpSum(y[(i,t)] for t in range(B)) <= d[i]
    
#No more than one task each block
for t in range(B):
    prob += lpSum(y[(i,t)] for i in range(n)) <= 1


In [38]:
prob.solve()

1

In [43]:
print("Assignment accomplished!")
for i in range(n):
    for t in range(B):
        if y[(i,t)].varValue == 1:
            print("task {} block {}".format(i,t))
        

Assignment accomplished!
task 0 block 12
task 0 block 20
task 1 block 6
task 1 block 13
task 1 block 25
task 1 block 26
task 3 block 7
task 4 block 8
task 4 block 11
task 4 block 19
task 4 block 28
task 4 block 29
task 6 block 18
task 6 block 27


In [40]:
pulp.value(prob.objective) 

50.0

In [44]:
prob.writeLP('task_problem.lp')

[Block_(0,_0),
 Block_(0,_1),
 Block_(0,_10),
 Block_(0,_11),
 Block_(0,_12),
 Block_(0,_13),
 Block_(0,_14),
 Block_(0,_15),
 Block_(0,_16),
 Block_(0,_17),
 Block_(0,_18),
 Block_(0,_19),
 Block_(0,_2),
 Block_(0,_20),
 Block_(0,_21),
 Block_(0,_22),
 Block_(0,_23),
 Block_(0,_24),
 Block_(0,_25),
 Block_(0,_26),
 Block_(0,_27),
 Block_(0,_28),
 Block_(0,_29),
 Block_(0,_3),
 Block_(0,_4),
 Block_(0,_5),
 Block_(0,_6),
 Block_(0,_7),
 Block_(0,_8),
 Block_(0,_9),
 Block_(1,_0),
 Block_(1,_1),
 Block_(1,_10),
 Block_(1,_11),
 Block_(1,_12),
 Block_(1,_13),
 Block_(1,_14),
 Block_(1,_15),
 Block_(1,_16),
 Block_(1,_17),
 Block_(1,_18),
 Block_(1,_19),
 Block_(1,_2),
 Block_(1,_20),
 Block_(1,_21),
 Block_(1,_22),
 Block_(1,_23),
 Block_(1,_24),
 Block_(1,_25),
 Block_(1,_26),
 Block_(1,_27),
 Block_(1,_28),
 Block_(1,_29),
 Block_(1,_3),
 Block_(1,_4),
 Block_(1,_5),
 Block_(1,_6),
 Block_(1,_7),
 Block_(1,_8),
 Block_(1,_9),
 Block_(2,_0),
 Block_(2,_1),
 Block_(2,_10),
 Block_(2,_11)

Questions:
-Is it necessary to complete one task in one day when get started?
-How to make the tasks continuous once get started but doesn't need to finish on the same day if the time does not allow?
    +precedence constraints (Single Machine Scheduling): One task cannot start until another task starts
    +No overlap constraints: One task at a time

Observations:
-The tasks with highest scores would be attempted to fit into the schedule until either time blocks are ran out out the task is finished